In [1]:
%%capture
!pip install pydub pyht
!pip install -U g4f[all]

In [ ]:
import random
from pydub import AudioSegment
from io import BytesIO
import requests
from tqdm.notebook import tqdm
import IPython
import json

from pyht import Client
from pyht.client import TTSOptions
import re

import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters


import nest_asyncio
import g4f
nest_asyncio.apply()

import shutil
from tqdm.notebook import tqdm

# GPT

In [ ]:
def find_acitve_gpt():
  active_providers = []
  providers = dir(g4f.Provider)
  for provider in providers:
    provider = getattr(g4f.Provider, provider)
    try:
      response = g4f.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": "Tell me your model name"}],
      provider=provider
      )
      print(response)

      if response != '':
        active_providers.append(provider)
    except:
      continue
  return active_providers

In [ ]:
def prompt(text):
  response = g4f.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "I need you to translate from Russian to English and rewrite my text for 'speech text' for narrator and to make it more engaging, but you should sound natural and in casual style, also revise text for proper spelling and grammar. Ignore all tables and tasks."},
              {'role':'assistant','content':'I am a professional rewriter and I am ready to assist you with rewriting your text to ensure better flow and increased engagement. Additionally, I will thoroughly review the text to correct any spelling and grammar errors. Thank you for entrusting me with this task.'},
              {'role':'user','content':text}],
    provider=g4f.Provider.Liaobots
)

  return response

# Voice

In [2]:
def get_payload(text):
  payload = {
      "text": text,
      "voice": "s3://voice-cloning-zero-shot/4bc2279c-ec2e-471f-9369-faf5dac29321/original/manifest.json",
      "output_format": "mp3",
      "voice_engine": "PlayHT2.0",
      "temperature": 0.333,
      "seed": 42}
  return payload

In [ ]:
def get_headers(api,user_id):
  headers = {
    "accept": "text/event-stream",
    "content-type": "application/json",
    "AUTHORIZATION": api,
    "X-USER-ID": user_id}
  return headers

In [ ]:
def api_checker(text):

  for API in APIs:
    if len(text) + 1 < APIs[API]:
      api = API
      url = "https://api.play.ht/api/v2/tts"

      payload = {
          "text": "h",
          "voice": "s3://voice-cloning-zero-shot/d9ff78ba-d016-47f6-b0ef-dd630f59414e/female-cs/manifest.json",
          "output_format": "mp3",
          "voice_engine": "PlayHT2.0"
      }
      headers = {
          "accept": "text/event-stream",
          "content-type": "application/json",
          "AUTHORIZATION": api,
          "X-USER-ID": user_ids[api]}

      response = requests.post(url, json=payload, headers=headers)

      if response.status_code == 200:
        APIs[api] -= 1
        return api
      else:
        continue

In [ ]:
def TTS(text):
  #название файла
  path = text.split()[0]

  # Создаем токенизатор предложений с нашими параметрами
  punkt_param = PunktParameters()
  punkt_param.abbrev_types = set(abbreviations)
  tokenizer = PunktSentenceTokenizer(punkt_param)
  # Разбиваем текст на предложения
  sentences = tokenizer.tokenize(text)


  audio_files = []
  for sentence in tqdm(sentences):
    api = api_checker(sentence)
    APIs[api] -= len(sentence)

    #создание headers
    user_id = user_ids[api]
    headers = get_headers(api,user_id)

    # создаем настройки для синтеза голоса, параметр только предложение
    payload = get_payload(sentence)
    make_audio = requests.post(playht_url, json=payload, headers=headers)

    audio_url = make_audio.text.split('"url":"')[1].split('","duration"')[0]

    download_audio = requests.get(audio_url)
    audio_bytes = BytesIO(download_audio.content)
    audio_file = AudioSegment.from_file(audio_bytes)
    audio_files.append(audio_file)

  with open('/content/drive/MyDrive/Colab Notebooks/datasets/APIs.json', 'w') as f:
    json.dump(user_ids, f)

  return merge_audio(audio_files,path)

In [ ]:
def merge_audio(audio_files,path):

  # Создание сегмента тишины продолжительностью 1 секунда
  silence = AudioSegment.silent(duration=1)

  # Создание списка сегментов аудиофайлов
  audio_segments = [AudioSegment.from_wav(file) for file in audio_files]

  # Создание списка для склеенных сегментов
  merged_segments = []

  # Добавление первого сегмента
  merged_segments.append(audio_segments[0])

  # Добавление паузы и остальных аудиофайлов
  for segment in tqdm(audio_segments[1:]):
    merged_segments.extend([silence, segment])

  # Склеивание аудиофайлов
  merged_audio = sum(merged_segments)

  path = f"{path}.wav"
  # Сохранение склеенного аудиофайла
  merged_audio.export(path, format="wav")
  return path

In [11]:
!ssh-keygen -t ed25519 -C "ggwpshnik27@gmail.com"

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): file
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in file
Your public key has been saved in file.pub
The key fingerprint is:
SHA256:FfVXW6US8yNKMrNnzPsk+8XhU+Y8Fqel2fmve5cpMs8 ggwpshnik27@gmail.com
The key's randomart image is:
+--[ED25519 256]--+
|          ..+   =|
|           . = .+|
|        + o o =..|
|         X . o o |
|        S *   ..=|
|         o . o %+|
|          o . O+*|
|           B.o.+=|
|          ..*E++=|
+----[SHA256]-----+
